# Image Classification with Deep Learning

## Importing Libraries

In [7]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

np.random.seed(123)
%matplotlib inline

## Reading the data

In [2]:
# Create data generators with rescaling 
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory('chest_xray/train/', target_size=(64, 64), batch_size=5216)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory('chest_xray/test/', target_size=(64, 64), batch_size=624)
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory('chest_xray/val/', target_size=(64, 64))

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [3]:
# Retrieve a batch of training images and their corresponding labels
train_images, train_labels = next(train_generator)

# Retrieve a batch of test images and their corresponding labels
test_images, test_labels = next(test_generator)

# Retrieve a batch of validation images and their corresponding labels
val_images, val_labels = next(val_generator)

In [4]:
# Checking the shape of the train_images and train_labels
train_images.shape, train_labels.shape

((5216, 64, 64, 3), (5216, 2))

In [5]:
# Checking the shape of the test_images and test_labels
test_images.shape, test_labels.shape

((624, 64, 64, 3), (624, 2))

In [6]:
# Checking the shape of the val_images and val_labels
val_images.shape, val_labels.shape

((16, 64, 64, 3), (16, 2))

The label's shape indicates that this is a binary classification problem. The labels suggest that the images can only represent two classes: "normal" or "pneumonia," which implies that the goal is to determine whether a patient has pneumonia or not.

## Modeling the Data

In [10]:
# Flattening the train set to all for modeling
train_reshaped = train_images.reshape(5216, 64*64*3)

# Flattening the test set to all for modeling
test_reshape = test_images.reshape(624, 64*64*3)

# Flattening the validation set to all for modeling
val_reshape = val_images.reshape(16, 64*64*3)

In [11]:
# initialized the model
dense_model = Sequential()

# Input Layer
dense_model.add(Dense(64, activation='relu', input_shape=(64*64*3,)))
# Hidden Layer
dense_model.add(Dense(64, activation="relu"))
dense_model.add(Dense(32, activation="LeakyReLU"))
# Output Layer
dense_model.add(Dense(2, activation="softmax"))

dense_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                786496    
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 2)                 66        
                                                                 
Total params: 792802 (3.02 MB)
Trainable params: 792802 (3.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Compile the dense neural network model
dense_model.compile(
    optimizer="adam",
    loss= "binary_crossentropy",
    metrics=['accuracy']
)

In [13]:
# Creating a EarlyStopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=4, mode='min')

# Defining the path for saving checkpoints
checkpoint_filepath = 'dense_model.h5'

# Create a ModelCheckpoint callback to save the best model weights
model_checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_loss',  
    save_best_only=True, 
    mode='min',  
    verbose=1  
)

In [14]:
history = dense_model.fit(
    train_reshaped,
    train_labels,
    epochs=1000,
    callbacks=[early_stopping, model_checkpoint],
    validation_data=(val_reshape, val_labels),
    batch_size=32
)

Epoch 1/1000
161/163 [============================>.] - ETA: 0s - loss: 0.2844 - accuracy: 0.8857
Epoch 1: val_loss improved from inf to 0.33863, saving model to dense_model.h5
163/163 [==============================] - 6s 25ms/step - loss: 0.2838 - accuracy: 0.8863 - val_loss: 0.3386 - val_accuracy: 0.9375
Epoch 2/1000
163/163 [==============================] - ETA: 0s - loss: 0.1631 - accuracy: 0.9402
Epoch 2: val_loss did not improve from 0.33863
163/163 [==============================] - 4s 24ms/step - loss: 0.1631 - accuracy: 0.9402 - val_loss: 1.2448 - val_accuracy: 0.5625
Epoch 3/1000
163/163 [==============================] - ETA: 0s - loss: 0.1621 - accuracy: 0.9413
Epoch 3: val_loss did not improve from 0.33863
163/163 [==============================] - 4s 25ms/step - loss: 0.1621 - accuracy: 0.9413 - val_loss: 0.3956 - val_accuracy: 0.8125
Epoch 4/1000
163/163 [==============================] - ETA: 0s - loss: 0.1477 - accuracy: 0.9457
Epoch 4: val_loss improved from 0.33863